In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [5]:
#Se renombra la base de Training a Df para el desarrollo del proyect
df = dataTraining

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7895 entries, 3107 to 215
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    7895 non-null   int64  
 1   title   7895 non-null   object 
 2   plot    7895 non-null   object 
 3   genres  7895 non-null   object 
 4   rating  7895 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 370.1+ KB


In [7]:
# busqueda valorres perdidos o NA
df.isna().any()

year      False
title     False
plot      False
genres    False
rating    False
dtype: bool

In [8]:
df['genres'].value_counts()

['Drama']                                                  429
['Comedy']                                                 368
['Comedy', 'Drama', 'Romance']                             306
['Comedy', 'Romance']                                      291
['Comedy', 'Drama']                                        287
                                                          ... 
['Adventure', 'Comedy', 'Family', 'Musical', 'Romance']      1
['Animation', 'Short', 'Comedy', 'Drama', 'War']             1
['Comedy', 'Mystery', 'Romance', 'Thriller']                 1
['Animation', 'Comedy', 'Family', 'Fantasy', 'Mystery']      1
['Animation', 'Comedy', 'Drama', 'Fantasy']                  1
Name: genres, Length: 1336, dtype: int64

In [9]:
df['rating'].describe()

count    7895.000000
mean        6.402812
std         1.078260
min         1.200000
25%         5.800000
50%         6.500000
75%         7.200000
max         9.300000
Name: rating, dtype: float64

In [10]:
# Descargar los recursos necesarios de NLTK
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\afval\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\afval\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
# Preprocesamiento de texto
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    words = text.lower().split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['plot'] = df['plot'].apply(preprocess_text)

In [12]:
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = text.strip(' ')
    return text

df['plot'] = df['plot'].apply(clean_text)

In [13]:
# Convertir los géneros en una representación binaria
df['genres'] = df['genres'].map(lambda x: eval(x))
mlb = MultiLabelBinarizer()
genres_binary = mlb.fit_transform(df['genres'])
genres_labels = mlb.classes_

In [14]:
genres_labels

array(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',
       'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War',
       'Western'], dtype=object)

In [15]:
vectorizer = CountVectorizer(max_features=1000, lowercase =True, stop_words='english', token_pattern=r'[^a-zA-Z]', analyzer='word', ngram_range=(1, 3), max_df=1.0, min_df=1)
X = vectorizer.fit_transform(df['plot'])

In [16]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, genres_binary, test_size=0.33, random_state=42)

In [17]:
rf = OneVsRestClassifier(RandomForestClassifier(min_samples_leaf= 4,n_jobs=-1, n_estimators=100, max_depth=10, random_state=42))
#rf = OneVsRestClassifier(RandomForestClassifier(class_weight = 'balanced',random_state = 42, max_depth = 10, min_samples_leaf= 4, min_samples_split= 10, n_estimators= 100))
rf.fit(X_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(max_depth=10,
                                                     min_samples_leaf=4,
                                                     n_jobs=-1,
                                                     random_state=42))

In [18]:
# Realizar predicciones en el conjunto de prueba
y_pred = rf.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test, y_pred, average='macro')

0.6049511963521629